In [1]:
# Download NLTK data
import nltk
nltk.download('stopwords')

# Check if the required packages are installed
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from nltk.corpus import stopwords
    import pandas as pd
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn.svm import LinearSVC
    import re
    # Import necessary libraries
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import StratifiedKFold
    import numpy as np
    print("All packages are successfully installed.")
except ImportError as e:
    print(f"Error: {e}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


All packages are successfully installed.


1. [20 pts] Consider the following cartoon (ref: https://xkcd.com/1443/)
Inspect the speech balloon and report any grammatical errors?

I don't mean to go all language nerd on you, but I just legit adverbed "legit", verbed "adverb", and adjectived "language nerd".

What was done here was to take various parts of the sentence and break it down into POS in unconventional ways. First, Legit was turned into an adverb, Language Nerd was turned into an Adjective, and Adverb was turned into a verb. It is unlikely that there are any grammatical errors. This is because while the sentence does use words uncommonly used for certain parts of speech, it is still grammatically correct.

2. [20 pts] Support your answer in (1.) by a grammar parser? Such as in the following. (Hint: You may need to add some rules)
Consider some news headlines in (Q3.) and (Q4.):

In [2]:
from nltk import CFG, ChartParser
grammar = CFG.fromstring("""
S -> NP Aux VP
S -> NP VP | VP
VP -> V | V NP | V NP PP | V PP | V Adv | V NP Adv | V NP PP Adv | V PP Adv
NP -> Pron
NP -> N | Det N | Det AP N | AP N | N PP | Det N PP | Det AP N PP | AP N PP
Pron -> 'I' | 'you'
Aux -> 'dont'
Adj -> 'all'
V -> 'mean' | 'go' | 'adverbed' | 'verbed' | 'adjectived'
VP -> VP 'to'
S -> S Conj S | Conj S | S Conj
Conj -> 'but' | 'and' | ','
Adv -> 'legit' | 'language-nerd'
VP -> VP VP | V AP PP | Adv VP
AP -> Adj | Adv Adj | Adj PP | Adv Adj PP
AP -> Adj Adv
VP -> VP VP | V AP PP | Adv VP
Prt -> 'on'
PP -> Prt | Prt NP
Rb -> 'just'
VP -> Rb VP
Conj -> 'but' | 'and' | ','
N -> 'legit' | 'adverb' | 'language' | 'nerd'
""")
from nltk import ChartParser
from nltk import CFG
c_parser = ChartParser(grammar)
Sentence ='I dont mean to go all language-nerd on you , but I just legit adverbed legit , verbed adverb , and adjectived language-nerd'.split(' ')


#I just legit adverbed legit , verbed adverb , and adjectived language-nerd
# to go all language nerd on you , but I just legit adverbed legit , verbed adverb , and adjectived language-nerd
# Generate the trees and display
trees = c_parser.parse(Sentence)
for tree in trees:
    tree.pretty_print()

                                                                         S                                                                                                  
                                                                _________|__________________________________________________________________________________                 
                                                               S                                                                            |               |               
                             __________________________________|___________________________________________                                 |               |                
                            |                                       |                                      S                                |               |               
                            |                                       |                           ___________|_________________________

3. [20 pts] Find out and explain what is wrong with the sentence "British Left Waffles on Falkland Islands" using a POS tagger.

In [36]:
import nltk
Sentence= "British Left Waffles on Falkland Islands"
tokens = nltk.word_tokenize(Sentence)
pos_tags= nltk.pos_tag(tokens)
for word in pos_tags:
    k=nltk.help.upenn_tagset(word[1])
    if k != None:
        print(k)

JJ: adjective or numeral, ordinal
    third ill-mannered pre-war regrettable oiled calamitous first separable
    ectoplasmic battery-powered participatory fourth still-to-be-named
    multilingual multi-disciplinary ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
IN: preposition or conjunction, subordinating
    astride among uppon whether out inside pro despite on by throughout
    below within for towards near behind atop around if like until below
    next into if beside ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer 

According to this POS tagger, we have a determiner followed by two proper nouns, a preposition, and two more proper nouns. Due to the lack of a verb phrase, this sentence is incorrect. Still, this sentence is incorrect due to the confusing context of this sentence. "British" is actually a noun in this context since they are the ones committing the action. Next, left is a verb, not a proper noun, since the British are the ones doing the leaving. Once these two part of speeches are changed, the sentence is gramatically correct.

4. [20 pts] What about the sentence "Juvenile Court to Try Shooting Defendant" using a POS tagger?

In [39]:
import nltk
Sentence= "Juvenile Court to Try Shooting Defendant"
tokens = nltk.word_tokenize(Sentence)
pos_tags= nltk.pos_tag(tokens)
for word in pos_tags:
    k=nltk.help.upenn_tagset(word[1])
    if k != None:
        print(k)

NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
TO: "to" as preposition or infinitive marker
    to
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christ

4. [20 pts] What is wrong about the sentence "Juvenile Court to Try Shooting Defendant" using a POS tagger?

This sentence, much like the one before it, is confusing due to ambiguous context. For instance, while there is at least one verb here, the direct object for "shooting" is not entirely clear. If shooting was a verb, the court would be shooting at the defendant. If shooting was an adjective, then that would mean it would be used to describe the defendant, which makes more semantic sense, there would only be one verb here. Last, it is unlikely though that shooting is a noun as identified by the parser.

5. [20 pts] In each IMDB movie reviews sentiment category {0,1}, pick one sentence which is not grammatically correct and present them. Pick the grammar from the module or make yours. (Hint: You can cut a sentence in half to create a grammar error)